In [1]:
!pip install -qU langchain langchain-openai langchain-community langchain-experimental beautifulsoup4 unstructured tiktoken faiss-cpu sentence-transformers


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


# Advanced RAG 기법 실습 (Pre-retrieval & Post-retrieval)

### 실습 목표
이 실습에서는 RAG(Retrieval-Augmented Generation) 시스템의 Pre-retrieval와 Post-retrieval 단계에서 적용할 수 있는 다양한 고급 기법들을 구현하고 비교해볼 예정입니다.

- Pre-retrieval 단계 : 검색 과정 전에 쿼리를 최적화하거나 검색 방법을 개선하여 더 관련성 높은 문서를 회수하는 데 도움을 줍니다.
- Post-retrieval : 또한, 주로 검색된 문서를 정제하고 재정렬하여 LLM의 답변 생성에 더 관련성 높은 컨텍스트를 제공하는 데 중점을 둡니다.

## 1. 환경 설정 및 문서 준비
필요한 라이브러리를 설치하고 불러옵니다.

In [2]:
# 라이브러리 불러오기
from langchain.chains import LLMChain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import (
    CrossEncoderReranker,
    DocumentCompressorPipeline,
    EmbeddingsFilter,
    LLMChainExtractor,
)
from langchain.schema import Document
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

## 샘플 문서 로드
비교를 위해 의미 있는 내용이 충분히 담긴 문서를 로드합니다.  
여기서는 **한국지능정보사회진흥원**의 AI국가전략 보고서 중 "전국민 AI 일상화 실행 계획"을 사용하겠습니다.
 - https://www.nia.or.kr/site/nia_kor/ex/bbs/View.do?cbIdx=99952&bcIdx=27378&parentSeq=27378

In [3]:
import requests

# PDF 다운로드 URL
pdf_url = "https://www.nia.or.kr/common/board/Download.do?bcIdx=27378&cbIdx=99952&fileNo=1"
pdf_filename = "National_AI_Plan.pdf"

# 파일 저장
with open(pdf_filename, "wb") as f:
    f.write(requests.get(pdf_url).content)

print("PDF 파일 저장 완료:", pdf_filename)

PDF 파일 저장 완료: National_AI_Plan.pdf


In [4]:
loader = PyPDFLoader("National_AI_Plan.pdf")
documents = loader.load()

## 문서를 잘 읽고 있는지 페이지를 달리하면서, 확인해보세요
print(documents[15].page_content[:500])

-6-
 Ⅲ. 국내 현황 진단◈전 세계적 디지털 모범국가로 나아가기 위한 정부의 전략 추진과민간의 선제적 혁신으로 전국민 AI일상화초석 마련○(정부)디지털 심화 시대에 대응한 신속한 전략･
정책추진을 통해 AI기술･
산업 경쟁력을 강화하고 민간 혁신을 뒷받침  * 대한민국 디지털 전략(‘22.9) → AI일상화·산업 고도화계획(’23.1)→초거대 AI경쟁력강화방안(‘23.4)-｢
AI법제정비단(’23.8~)｣
를 통해 AI확산에 따른 사회적 이슈를 정비하고,AI거버넌스 정립과 공동 번영을 위한 글로벌 협력 본격화○(민간)독자적 초거대 AI개발･
출시 본격화,중소･
스타트업 중심의다양한 응용서비스출시 등 산업 생태계 조성시작네이버, ‘클로바’LG, ‘엑사원’카카오, KoGPT SKT, ‘에이닷’KT, ‘믿음’
-의료 등 다양한 도메인에서의중소·스타트업 역시 초거대 AI기반전문서비스를 출시,체감되는 AI활용·확산을 주도
* 의료 AI 기업 ‘루닛’이 美정부의 암정복 프로젝트 ‘캔서 문샷


In [5]:
# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

print(f"총 문서 개수: {len(chunks)}")

총 문서 개수: 99


## 2. 기본 RAG 체인 구현 (Naive RAG)
가장 기본적인 RAG 체인을 구현합니다. 다른 기법들과 비교할 기준점이 됩니다.  

모델은 `gpt-4o-mini`을 사용합니다. 가볍고 비용이 저렴하며 성능 또한 우수하기에 모델의 기본 성능에 가려져 여러 기법 적용 시 큰 차이를 느끼지 못할 수 있습니다. 그러나 제한된 환경에서 오픈소스 LLM을 사용한다면 LLM 외 파이프라인 구조에서 더 큰 성능의 차이를 느낄 수 있습니다. 이번 실습은 여러 방식에 대한 소개를 중점으로 진행합니다.

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일 로드, 환경 변수에서 API 키 읽기
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [9]:
## OPENAI 임베딩 모델이 있지만 비용 문제로, 다른 모델 선택.
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

#  한국어 특화 SentencePiece 기반 임베딩 모델 (SemanticChunker 및 VectorStore 생성에 사용)
# model_name = "jhgan/ko-sroberta-multitask"
# embeddings = HuggingFaceEmbeddings(model_name=model_name)

In [10]:
# FAISS 벡터 스토어 생성
vectorstore = FAISS.from_documents(chunks, embeddings)

# 기본 검색기 생성 (k=3)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# LLM 초기화
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, openai_api_key=os.environ["OPENAI_API_KEY"])

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_template("""
다음 정보를 사용하여 사용자의 질문에 답변하세요.
정보에 관련 내용이 없으면 "제공된 정보에서 답을 찾을 수 없습니다"라고 말하세요.

[정보]:
{context}

[질문]:
{question}

[답변]:
""")

# Document Chain 생성
document_chain = create_stuff_documents_chain(llm, prompt)

# Naive RAG 체인 구성 (파이프라인 방식)
naive_rag_chain = {"context": retriever, "question": RunnablePassthrough()} | document_chain


def naive_rag(query):
    """기본 RAG 체인으로 답변을 생성합니다."""
    result = naive_rag_chain.invoke(query)
    return result

테스트 쿼리를 실행해 Naive RAG의 작동을 확인합니다.

In [11]:
# 테스트 쿼리로 Naive RAG 실행
test_queries = [
    "AI 일상화를 위한 핵심 전략은 무엇인가요?",
    "복지 분야에서 AI는 어떤 방식으로 활용되나요?",
    "AI를 활용한 행정 혁신 방안은 무엇인가요?",
    "AI 디지털 교과서는 언제 도입될 예정인가요?",
]

for i, query in enumerate(test_queries, 1):
    print(f"\n테스트 {i}: {query}")
    print("-" * 50)

    print("Naive RAG 답변:")
    print(naive_rag(query))
    print("-" * 30)


테스트 1: AI 일상화를 위한 핵심 전략은 무엇인가요?
--------------------------------------------------
Naive RAG 답변:
제공된 정보에서 답을 찾을 수 없습니다.
------------------------------

테스트 2: 복지 분야에서 AI는 어떤 방식으로 활용되나요?
--------------------------------------------------
Naive RAG 답변:
복지 분야에서 AI는 주로 AI 챗봇과 AI 돌봄 로봇을 통해 활용됩니다. 이러한 기술들은 국민의 복지 서비스를 개선하고, 보다 효율적인 지원을 제공하는 데 기여하고 있습니다.
------------------------------

테스트 3: AI를 활용한 행정 혁신 방안은 무엇인가요?
--------------------------------------------------
Naive RAG 답변:
제공된 정보에서 답을 찾을 수 없습니다.
------------------------------

테스트 4: AI 디지털 교과서는 언제 도입될 예정인가요?
--------------------------------------------------
Naive RAG 답변:
제공된 정보에서 답을 찾을 수 없습니다.
------------------------------


## 3. 쿼리 변환 기법 구현(Pre-retrieval)
### 3.1 쿼리 확장(Multi-query Expansion)

쿼리 확장은 원래 쿼리의 의도와 핵심 개념을 파악한 후, 의미는 동일하지만 표현이 서로 다른 여러 개의 쿼리를 생성하는 방법입니다. 이렇게 하면 초기 쿼리로는 회수하지 못했을 관련 문서를 찾을 확률이 높아집니다.

In [12]:
# 쿼리 확장 프롬프트 정의
multi_query_prompt = PromptTemplate(
    input_variables=["question"],
    template="""
당신은 검색 정확도를 향상시키기 위해 다양하지만 의미적으로 관련된 쿼리를 생성하는 전문가입니다.

다음 사용자 질문에 대해 3개의 다른 표현 또는 관점을 반영한 대체 쿼리를 생성하세요.
Python 형식의 문자열 리스트로 반환하세요 (예: ["...", "...", "..."]).

사용자 질문:
"{question}"

확장된 쿼리:
""",
)

# 수동 쿼리 확장 체인 생성
multi_query_chain = LLMChain(llm=llm, prompt=multi_query_prompt)


def get_expanded_queries(question):
    """사용자 질문을 여러 확장 쿼리로 변환합니다."""
    result = multi_query_chain.run(question=question)
    try:
        # 결과를 Python 리스트로 평가
        queries = eval(result)
        # 원래 쿼리 추가
        queries.append(question)
        return queries
    except:
        # 평가 실패 시 기본값 반환
        print("쿼리 확장 결과 평가에 실패했습니다. 원본 쿼리만 사용합니다.")
        return [question]


def multi_query_retrieval(question, vectorstore, k=3):
    """여러 확장 쿼리를 사용해 문서를 검색합니다."""
    # 쿼리 확장
    expanded_queries = get_expanded_queries(question)
    print("생성된 확장 쿼리:")
    for i, query in enumerate(expanded_queries):
        print(f"  {i + 1}. {query}")

    # 각 쿼리로 문서 검색
    all_docs = []
    for query in expanded_queries:
        docs = vectorstore.similarity_search(query, k=k)
        all_docs.extend(docs)

    # 중복 제거
    unique_docs = []
    unique_contents = set()
    for doc in all_docs:
        if doc.page_content not in unique_contents:
            unique_docs.append(doc)
            unique_contents.add(doc.page_content)

    # 상위 k개 문서만 반환 (더 많은 문서를 반환할 수도 있음)
    return unique_docs[: k * 2]  # 일반적으로 더 많은 문서를 사용

/tmp/ipykernel_150/4200574479.py:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  multi_query_chain = LLMChain(llm=llm, prompt=multi_query_prompt)


In [13]:
# 멀티 쿼리 확장 RAG 비교 실행
test_query = "AI를 활용한 행정 혁신 방안은 무엇인가요?"
print("\n" + "=" * 50)
print(f"\n질문: {test_query}")
print("\n" + "=" * 50)

# 문서 검색 및 문서 내용 표시
expanded_docs = multi_query_retrieval(test_query, vectorstore)
print(f"\n검색된 문서 수: {len(expanded_docs)}")
print("검색된 문서 내용 (처음 2개):")
for i, doc in enumerate(expanded_docs[:2]):
    print(f"\n문서 {i + 1}:")
    print(doc.page_content[:200] + "..." if len(doc.page_content) > 200 else doc.page_content)

# 멀티 쿼리 확장 RAG 체인 실행
multi_query_result = document_chain.invoke(
    {
        "context": expanded_docs,  # ← 문자열이 아니라 List[Document]
        "question": test_query,
    }
)

print("\nNaive RAG 답변:")
print(naive_rag(test_query))

print("\n멀티 쿼리 확장 RAG 답변:")
print(multi_query_result)



질문: AI를 활용한 행정 혁신 방안은 무엇인가요?



/tmp/ipykernel_150/4200574479.py:22: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = multi_query_chain.run(question=question)


쿼리 확장 결과 평가에 실패했습니다. 원본 쿼리만 사용합니다.
생성된 확장 쿼리:
  1. AI를 활용한 행정 혁신 방안은 무엇인가요?

검색된 문서 수: 3
검색된 문서 내용 (처음 2개):

문서 1:
-8-
-이미 AI을 도입한 기업들 역시 적용 대상·활용 수준이 모두 제한적으로 전면 도입·확산은 여전한 과제 AI 도입 수준 (SPRI, ‘22)AI 서비스 활용현황 (SPRI, ‘22)

문서 2:
활용,AI를 누구나 쉽게 체감할 수 있는 AI대중화 시대에 본격 진입   * (예시) 스마트폰의 AI비서 및 사진보정, OTT 플랫폼의 개인 취향분석 콘텐츠 추천 등○산업현장･
일터에서도 AI가 노동의 양을 줄이고,질은 높이는 생산성·효율성 제고*의 원천으로 빠르게 확산 중   * (예) 짧은 보고서 작성에 챗GPT 활용 시 약 37% 작업시간 단축(MIT...

Naive RAG 답변:
제공된 정보에서 답을 찾을 수 없습니다.

멀티 쿼리 확장 RAG 답변:
제공된 정보에서 답을 찾을 수 없습니다.


### 3.2 HyDE(Hypothetical Document Embedding)
HyDE는 쿼리를 바로 임베딩하여 검색하는 대신, LLM을 통해 쿼리에 대한 예상 답변(가설 문서)을 먼저 생성하고, 그 가설 문서를 임베딩하여 유사한 실제 문서를 찾는 방법입니다. 이를 통해 질문과 비슷한 문서가 아닌 답변과 비슷한 문서를 회수할 수 있습니다.  

우선 HyDE 작업을 수행하기 위하여 프롬프트를 정의합니다. 아래 프롬프트는 기본적인 기능만 수행하므로, 실험 결과를 개선하기 위하여 추가로 수정해주셔도 좋습니다.

In [14]:
# HyDE 프롬프트 정의
hyde_prompt = PromptTemplate(
    input_variables=["query"],
    template="""
당신은 사용자의 질문에 대한 가상의 문서를 생성하는 전문가입니다.
질문에 대한 답변이 될 수 있는 가상의 문서 내용을 생성하세요.
이 가상 문서는 실제 검색에 사용될 것이므로 명확하고 정보가 풍부해야 합니다.

사용자 질문:
{query}

가상 문서:""",
)

# 가상 문서 생성 Chain
hyde_chain = LLMChain(llm=llm, prompt=hyde_prompt)

이어 가상 문서 생성을 위한 함수를 선언합니다.

In [15]:
def generate_hypothetical_documents(query, num_docs=3):
    """주어진 쿼리에 대해 여러 개의 가상 문서를 생성합니다."""
    hypothetical_docs = []

    for i in range(num_docs):
        hyp_doc = hyde_chain.run(query)
        hypothetical_docs.append(hyp_doc)

    return hypothetical_docs


def retrieve_with_hyde(query, vectorstore, num_hypothetical_docs=3, docs_per_hyp=2):
    """HyDE 기법을 사용하여 문서를 검색합니다."""
    # 가상 문서 생성
    hyp_docs = generate_hypothetical_documents(query, num_hypothetical_docs)
    print("생성된 가상 문서:")
    for i, doc in enumerate(hyp_docs):
        print(f"\n가상 문서 {i + 1}:")
        print(doc[:200] + "..." if len(doc) > 200 else doc)

    # 각 가상 문서로 검색 수행
    all_retrieved_docs = []
    for i, hyp_doc in enumerate(hyp_docs):
        # 가상 문서를 임베딩하고 유사한 문서 검색
        retrieved_docs = vectorstore.similarity_search(hyp_doc, k=docs_per_hyp)
        print(f"\n가상 문서 {i + 1}로 검색된 실제 문서:")
        for j, doc in enumerate(retrieved_docs):
            print(f"  문서 {j + 1}: {doc.page_content[:100]}...")
        all_retrieved_docs.extend(retrieved_docs)

    # 중복 제거 (동일한 내용이 있는 경우)
    unique_docs = []
    unique_contents = set()

    for doc in all_retrieved_docs:
        if doc.page_content not in unique_contents:
            unique_docs.append(doc)
            unique_contents.add(doc.page_content)

    return unique_docs

테스트용 쿼리를 만든 후, 이를 바탕으로 가상 문서를 생성하여 이와 유사한 문서를 검색합니다.

In [16]:
# HyDE RAG 비교 실행
hyde_query = "산업 생산 구조에서 AI의 동향에 대해 알려주세요."
print("\n" + "=" * 50)
print(f"\n질문: {hyde_query}")

# HyDE로 문서 검색
hyde_docs = retrieve_with_hyde(hyde_query, vectorstore)
print(f"\n최종 검색된 문서 수: {len(hyde_docs)}")



질문: 산업 생산 구조에서 AI의 동향에 대해 알려주세요.
생성된 가상 문서:

가상 문서 1:
# 산업 생산 구조에서 AI의 동향

## 서론
인공지능(AI)은 현대 산업 생산 구조의 혁신을 이끌고 있으며, 다양한 분야에서 효율성을 극대화하고 있습니다. 이 문서에서는 AI의 주요 동향, 적용 사례, 그리고 산업 생산 구조에 미치는 영향을 살펴보겠습니다.

## 1. AI의 발전과 산업 생산 구조
AI 기술은 머신러닝, 딥러닝, 자연어 처리(NLP) ...

가상 문서 2:
# 산업 생산 구조에서 AI의 동향

## 서론
인공지능(AI)은 현대 산업 생산 구조의 혁신을 이끌고 있으며, 다양한 분야에서 효율성을 높이고 비용을 절감하는 데 기여하고 있습니다. 본 문서에서는 AI의 주요 동향, 적용 사례, 그리고 산업 생산 구조에 미치는 영향을 살펴보겠습니다.

## 1. AI의 주요 동향

### 1.1 자동화 및 로봇 공학
AI...

가상 문서 3:
# 산업 생산 구조에서 AI의 동향

## 서론
인공지능(AI)은 현대 산업 생산 구조의 혁신을 이끌고 있으며, 다양한 분야에서 효율성을 높이고 비용을 절감하는 데 기여하고 있습니다. 본 문서에서는 AI의 주요 동향, 적용 사례, 그리고 산업 생산 구조에 미치는 영향을 살펴보겠습니다.

## 1. AI의 주요 동향

### 1.1 자동화 및 로봇 공학
AI...

가상 문서 1로 검색된 실제 문서:
  문서 1: ⇨일상 모든 곳에서 전 국민이보다 쉽게 AI를 활용,더 편리한 삶을 누릴 수 있도록AI활용의 양적·질적 확대가 필요 □ (일터) 생산성 제고를 위한 AI 활용 본격화, 전면 확산이...
  문서 2: -18-
 <2-4> 산업 생산구조를 혁신하는 AI 확산 가속화 ◇ 기존산업에 AI 내재화를 통해 새로운 성장 동력을 마련하고, 디지털 경제 시대를 선도할 수 있는 산업 구조로 전...

가상 문서 2로 검색된 실제 문서:
  문서 1: ⇨일상 모든 곳에서 전 국민이보다 쉽게 AI를 활용,더 편리한 삶을 

가상 문서를 통해 추가로 검색된 문서들을 바탕으로 답변을 생성합니다. 이를 Naive RAG와 비교해봅시다.

In [17]:
# HyDE RAG 체인 실행
hyde_result = document_chain.invoke(
    {
        "question": hyde_query,
        "context": hyde_docs,  # ← List[Document] 객체
    }
)

print("TEST QUERY : ", hyde_query)
print("==" * 50)
print("\nNaive RAG 답변:")
print(naive_rag(hyde_query))

print("\nHyDE RAG 답변:")
print(hyde_result)

TEST QUERY :  산업 생산 구조에서 AI의 동향에 대해 알려주세요.

Naive RAG 답변:
산업 생산 구조에서 AI의 동향은 다음과 같습니다. AI는 기존 산업에 내재화되어 새로운 성장 동력을 마련하고, 디지털 경제 시대를 선도할 수 있는 산업 구조로의 전환을 가속화하고 있습니다. 특히, 산업현장에 최적화된 AI 적용을 지원하기 위해 AI 기업이 보유한 다양한 솔루션(제품 설계, 설비 진단, 불량 검출 등)을 수요 기업에 맞춤형으로 제공하고 있습니다.

AI의 활용은 일터에서 생산성 제고를 위한 본격적인 도입으로 이어지고 있으며, 단순하고 반복적인 업무에 AI를 적극적으로 도입함으로써 노동시간 단축과 업무 효율성 향상 등의 효과를 빠르게 체감하고 있습니다. 예를 들어, SK하이닉스는 AI를 통해 에너지 비용을 절감하는 성과를 거두었고, 인트플로우와 LG스마트파크는 각각 돼지 생체정보 관리와 공장 관리에 AI를 도입하여 생산성을 크게 향상시켰습니다.

그러나 국내 기업의 AI 도입률은 여전히 낮은 수준(14.7%)이며, 산업 분야별로 AI 도입 격차가 존재하는 상황입니다. 공공·안전 분야는 23.7%, 교통·물류는 17.8%의 도입 비율을 보이는 반면, 제조업은 9.3%, 의료는 8.5%로 상대적으로 낮은 수치를 기록하고 있습니다. 이러한 동향은 AI의 양적·질적 확대가 필요함을 시사합니다.

HyDE RAG 답변:
산업 생산 구조에서 AI의 동향은 다음과 같습니다. AI의 활용이 기존 산업에 내재화되어 새로운 성장 동력을 마련하고, 디지털 경제 시대를 선도할 수 있는 산업 구조로의 전환이 가속화되고 있습니다. 특히, 국내 산업현장과 일터에서는 단순하고 반복적인 업무에 AI를 적극 도입하여 노동시간을 단축하고 업무 효율성을 향상시키는 효과를 빠르게 체감하고 있습니다. 예를 들어, SK하이닉스는 설비 운영에 AI를 도입하여 2022년 기준으로 142억원의 에너지 비용을 절감하였고, LG스마트파크는 AI를 통해 공장 생산성을 20% 향상시켰습니다. 이러

## 4. 컨텍스트 필터링(Context Filtering) 구현
컨텍스트 필터링은 검색된 문서 중 불필요하거나 관련성이 낮은 정보를 제거하는 기법입니다. 여기서는 유사도 기반 필터링을 구현하여, 임계값 이하의 유사도를 가진 문서를 제거합니다.

In [18]:
# 유사도 기반 필터링 구현
def get_similarity_scores(query, k=15):
    """쿼리에 대한 문서들의 유사도 점수를 검색합니다."""
    docs_and_scores = vectorstore.similarity_search_with_score(query, k=k)
    return docs_and_scores


def filter_by_similarity_threshold(docs_and_scores, threshold=1.1):
    """
    거리(distance) 기반 필터링:
    거리 값이 threshold 이하인, 즉 더 가까운(유사한) 문서만 통과시킵니다.
    (값이 작을수록 더 유사함)
    """
    filtered_docs = [doc for doc, score in docs_and_scores if score <= threshold]
    return filtered_docs


# 임베딩 필터 생성
embeddings_filter = EmbeddingsFilter(
    embeddings=embeddings,
    similarity_threshold=0.7,  # 유사도(similarity): 클수록 더 유사함
)

# 필터 검색기 생성
filter_retriever = ContextualCompressionRetriever(
    base_retriever=vectorstore.as_retriever(search_kwargs={"k": 15}), base_compressor=embeddings_filter
)

# 필터링된 RAG 체인 구성
filter_rag_chain = {"context": filter_retriever, "question": RunnablePassthrough()} | document_chain


def filter_rag(query):
    """필터링을 적용한 RAG 체인으로 답변을 생성합니다."""
    result = filter_rag_chain.invoke(query)
    return result

임계값을 조정하면서 필터링 효과를 테스트해 봅니다.

In [19]:
# 유사도 점수 확인 및 필터링 테스트
test_query = "AI 일상화를 위한 핵심 전략은 무엇인가요?"
docs_and_scores = get_similarity_scores(test_query, k=15)

print("검색된 문서의 유사도 점수:")
for i, (doc, score) in enumerate(docs_and_scores):
    print(f"문서 {i + 1} - 점수: {score:.4f}")
    print(f"내용 미리보기: {doc.page_content[:100]}...\n")

print("=" * 75)

# 다양한 임계값으로 필터링 테스트
thresholds = [1.2, 1.1, 1.0]
for threshold in thresholds:
    filtered_docs = filter_by_similarity_threshold(docs_and_scores, threshold)
    print(f"\n임계값 {threshold}로 필터링 후 남은 문서 수: {len(filtered_docs)}")

# 필터링된 RAG 실행
print("\n필터링 RAG 답변:")
print(filter_rag(test_query))

# Naive RAG와 비교
print("\nNaive RAG 답변:")
print(naive_rag(test_query))

검색된 문서의 유사도 점수:
문서 1 - 점수: 1.0316
내용 미리보기: 전국민 AI 일상화 실행계획
,
2023. 9.
관 계 부 처  합 동...

문서 2 - 점수: 1.0651
내용 미리보기: -i-
전국민 AI 일상화 실행계획(안) (요약)Ⅰ추진 배경□새정부 출범이후 ‘뉴욕구상(’22.9)’,‘대한민국 디지털 전략(’22.9)’을통해 국민과 함께 세계의 모범이 되는 디...

문서 3 - 점수: 1.0682
내용 미리보기: 일터및공공분야까지AI일상화 조기 실현을 위한 범부처 역량 결집○국민 누구나 일상 속에서 AI혜택을 누릴 수 있도록 하고,이를 대규모 AI수요 창출로 연결하여 AI산업 육성추진 ⇨A...

문서 4 - 점수: 1.0978
내용 미리보기: -iii-
Ⅴ주요 내용 전략1. AI로 국민 일상을 풍요롭게 하겠습니다.□(복지) 독거노인,보호아동,장애인 등 사회적 약자 배려･
돌봄대상내용부처독거노인전국 보건소(261개소)를 ...

문서 5 - 점수: 1.1296
내용 미리보기: 유해성 표현등초거대 AI한계 극복을 위한 기술개발 신규 추진(’24~)...

문서 6 - 점수: 1.1373
내용 미리보기: -2-
참고  ｢전국민 AI 일상화 실행계획｣ 추진경과□ ｢대한민국 디지털 전략｣ 및 ｢新성장 4.0 전략｣ 수립ㅇ‘뉴욕구상’의 기조･
철학을 반영하여,AI등 디지털 역량 강화,포...

문서 7 - 점수: 1.1590
내용 미리보기: 순    서   Ⅰ. 추진 배경 1 Ⅱ. AI 일상화 시대의 본격 개막4 Ⅲ. 국내 현황 진단6 Ⅳ. 비전 및 중점 추진과제 9 Ⅴ. 추진과제 10  1. AI로 국민 일상을 풍요...

문서 8 - 점수: 1.1619
내용 미리보기: -27-
 Ⅵ. 추진체계 및 계획□ 추진체계 및 향후계획ㅇ ’정보통신전략위원회‘를 통해 동 실행계획의 추진실적 점검 ㅇ 각 부처 소관 영역의 AI일상화 신규과제 발굴,예산 반영 협...

문서 9 - 점수: 1.1685
내용 미리보기: -6-
 Ⅲ. 국내 현황 진단◈전 세계적 

## 5. 컨텍스트 압축(Context Compression) 구현
컨텍스트 압축은 검색된 문서의 핵심 정보를 유지하면서 크기를 줄이는 기법입니다. 여기서는 LLM을 활용한 생성형 요약 방식을 구현합니다.

In [20]:
# LLM을 활용한 문서 요약 구현
summarization_prompt = PromptTemplate(
    template="""다음 텍스트를 주요 정보를 유지하면서 간결하게 요약해주세요:

{text}

요약:""",
    input_variables=["text"],
)

summarize_chain = LLMChain(llm=llm, prompt=summarization_prompt)


def summarize_documents(docs):
    """문서 목록을 요약합니다."""
    summarized_docs = []
    for doc in docs:
        summary = summarize_chain.run(text=doc.page_content)
        # 원본 메타데이터 유지하면서 내용만 요약으로 교체
        summarized_doc = Document(page_content=summary, metadata=doc.metadata)
        summarized_docs.append(summarized_doc)
    return summarized_docs


# LLM 기반 문서 압축기 생성
llm_compressor = LLMChainExtractor.from_llm(llm)

# 압축 검색기 생성
compression_retriever = ContextualCompressionRetriever(
    base_retriever=vectorstore.as_retriever(search_kwargs={"k": 5}), base_compressor=llm_compressor
)

# 압축 RAG 체인 구성
compression_rag_chain = {"context": compression_retriever, "question": RunnablePassthrough()} | document_chain


def compression_rag(query):
    """압축을 적용한 RAG 체인으로 답변을 생성합니다."""
    result = compression_rag_chain.invoke(query)
    return result

압축된 문서를 사용한 RAG 체인의 성능을 테스트합니다.

In [21]:
# 테스트 쿼리로 문서 검색
test_query = "AI 일상화를 위한 핵심 전략은 무엇인가요?"
retrieved_docs = vectorstore.similarity_search(test_query, k=5)

# 검색된 문서 요약
summarized_docs = summarize_documents(retrieved_docs)

print("원본 문서와 요약 문서 비교:")
for i, (orig_doc, summ_doc) in enumerate(zip(retrieved_docs, summarized_docs, strict=False)):
    print(f"\n문서 {i + 1}:")
    print(f"원본({len(orig_doc.page_content)} 자): {orig_doc.page_content[:150]}...")
    print("-" * 75)
    print(f"요약({len(summ_doc.page_content)} 자): {summ_doc.page_content}")

# 압축 RAG 실행
print("\n압축 RAG 답변:")
print(compression_rag(test_query))

# Naive RAG와 비교
print("\nNaive RAG 답변:")
print(naive_rag(test_query))

원본 문서와 요약 문서 비교:

문서 1:
원본(39 자): 전국민 AI 일상화 실행계획
,
2023. 9.
관 계 부 처  합 동...
---------------------------------------------------------------------------
요약(192 자): 전국민 AI 일상화 실행계획은 2023년 9월에 발표된 정부의 정책으로, AI 기술을 국민의 일상에 통합하기 위한 전략을 담고 있다. 주요 목표는 AI 활용을 통해 생활 편의성을 높이고, 경제적 효율성을 증대시키며, 교육 및 사회 서비스의 질을 향상시키는 것이다. 이를 위해 관련 부처가 협력하여 다양한 프로그램과 지원 방안을 마련할 예정이다.

문서 2:
원본(483 자): -i-
전국민 AI 일상화 실행계획(안) (요약)Ⅰ추진 배경□새정부 출범이후 ‘뉴욕구상(’22.9)’,‘대한민국 디지털 전략(’22.9)’을통해 국민과 함께 세계의 모범이 되는 디지털 강국실현 추진   * ‘파리 이니셔티브(’23.6)’를 통해 새로운 디지털 질서･규범...
---------------------------------------------------------------------------
요약(209 자): 전국민 AI 일상화 실행계획(안)은 새정부 출범 이후 디지털 강국 실현을 목표로 하며, AI 경쟁력 강화를 위한 다양한 정책을 추진하고 있습니다. AI의 일상화가 진행됨에 따라, 국민이 AI의 혜택을 공유하고 이를 잘 활용하는 디지털 모범국가로 도약할 시점이 도래했습니다. 초거대 AI의 등장으로 AI 활용이 쉬워지며, 국가의 AI 경쟁력은 AI 활용 능력에 달려 있습니다.

문서 3:
원본(160 자): 일터및공공분야까지AI일상화 조기 실현을 위한 범부처 역량 결집○국민 누구나 일상 속에서 AI혜택을 누릴 수 있도록 하고,이를 대규모 AI수요 창출로 연결하여 AI산업 육성추진 ⇨AI를 국민의 삶 전반에 전방위적으로 확산하고,디지털 모범국가의 탄탄한 기초로서 범부처 AI...
--

## 6. Reranking 구현
Reranking은 초기 검색 결과를 더 정교한 방법으로 재평가하여 문서 관련성을 재정렬하는 기법입니다. 여기서는 BERT 기반 Cross-Encoder를 활용한 Reranker를 구현합니다.

### Reranker
VectorDB의 검색 결과를 더 정교한 방법으로 재평가하여 쿼리와 문서 사이 관련성을 재정렬하는 기법을 말합니다. 머신러닝 기반 알고리즘, 딥러닝 모델 등을 활용하여 이 작업을 수행할 수 있으며 이번 실습에서는 BERT 기반 Cross-Encoder을 사용합니다.


### Cross-encoder
두 개의 입력 문장을 함께 처리하여 이들 간의 관계나 유사성을 평가하는 신경망 아키텍쳐입니다. 딥러닝 모델 중에서는 현재 BERT 기반 모델을 주로 사용합니다. 오픈소스 모델인 `bge-reranker`을 사용하여 쿼리와 검색된 문서 간 관련성을 재평가하겠습니다.

In [ ]:
# Cross-Encoder Reranker 구현
cross_encoder = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
reranker = CrossEncoderReranker(model=cross_encoder, top_n=3)

# Reranking 검색기 생성
reranking_retriever = ContextualCompressionRetriever(
    base_retriever=vectorstore.as_retriever(search_kwargs={"k": 10}), base_compressor=reranker
)

# Reranking RAG 체인 구성
reranking_rag_chain = {"context": reranking_retriever, "question": RunnablePassthrough()} | document_chain


def reranking_rag(query):
    """Reranking을 적용한 RAG 체인으로 답변을 생성합니다."""
    result = reranking_rag_chain.invoke(query)
    return result

Reranker를 적용한 검색 결과와 원래 검색 결과를 비교합니다.

In [ ]:
# 테스트 쿼리로 문서 검색
test_query = "정보통신전략위원회는 어떤 역할을 하나요?"
retrieved_docs = vectorstore.similarity_search(test_query, k=10)


# Cross-Encoder 점수 계산
def get_cross_encoder_scores(query, docs):
    """Cross-Encoder를 사용해 문서의 관련성 점수를 계산합니다."""
    text_pairs = [(query, doc.page_content) for doc in docs]
    scores = cross_encoder.score(text_pairs)
    return list(zip(docs, scores, strict=False))


# 원본 검색 결과와 Reranker 적용 결과 비교
reranked_docs = reranker.compress_documents(retrieved_docs, test_query)

print("원본 검색 결과:")
for i, doc in enumerate(retrieved_docs):
    print(f"\n문서 {i + 1}: {doc.page_content[:150]}...")

docs_with_scores = get_cross_encoder_scores(test_query, retrieved_docs)
docs_with_scores.sort(key=lambda x: x[1], reverse=True)

print("=" * 75)

print("\nCross-Encoder 점수로 재정렬된 결과 (상위 3개):")
for i, (doc, score) in enumerate(docs_with_scores[:3]):
    print(f"\n문서 {i + 1} (점수: {score:.4f}): {doc.page_content[:150]}...")

print("=" * 75)

# Reranking RAG 실행
print("\nReranking RAG 답변:")
print(reranking_rag(test_query))

print("=" * 75)

# Naive RAG와 비교
print("\nNaive RAG 답변:")
print(naive_rag(test_query))

## 7. 다양한 Post-retrieval 기법 조합 및 비교
여러 Post-retrieval 기법을 조합하여 더 나은 결과를 얻을 수 있습니다. 여기서는 필터링 + Reranking 조합을 구현합니다.

In [ ]:
def safe_compress_documents(compressor, docs, query):
    """
    # 검색된 문서가 없을 시 요약 작업을 수행하지 않음
    """
    if not docs:
        return []  # 검색된 문서가 없을 시 빈 리스트 반환
    try:
        return compressor.compress_documents(docs, query)
    except IndexError:
        print("No context found in vector store. Falling back to original documents.")
        return docs


# 필터링 + Reranking 조합 구현
compressor_pipeline = DocumentCompressorPipeline(
    transformers=[
        embeddings_filter,  # 먼저 유사도 기반 필터링
        reranker,  # 그 다음 Reranking
    ]
)

# 조합된 검색기 생성
combined_retriever = ContextualCompressionRetriever(
    base_retriever=vectorstore.as_retriever(search_kwargs={"k": 15}), base_compressor=compressor_pipeline
)

# 조합된 RAG 체인 구성
combined_rag_chain = {"context": combined_retriever, "question": RunnablePassthrough()} | document_chain


def combined_rag(query):
    """
    여러 기법을 조합한 RAG 체인으로 답변을 생성합니다.
    빈 리스트를 받으면 이를 그냥 반환하여 IndexError를 방지합니다
    """
    try:
        # First try with the pipeline
        result = combined_rag_chain.invoke(query)
        return result
    except Exception as e:
        print(f"No data relevant in VectorDB: {e}")
        print("유사도 기반 필터링 결과 활용할 수 있는 컨텍스트가 없습니다.")
        print("Compression과 Reranking을 사용하여 답변을 생성합니다.\n")
        # Fall back to reranking only
        return reranking_rag(query)

모든 방식의 RAG 체인을 비교해 봅니다.


In [ ]:
# 모든 RAG 방식 비교
test_query = "AI 일상화를 위한 핵심 전략은 무엇인가요?"

print("1. Naive RAG 답변:")
print(naive_rag(test_query))
print("\n" + "=" * 70 + "\n")

print("2. 필터링 RAG 답변:")
print(filter_rag(test_query))
print("\n" + "=" * 70 + "\n")

print("3. 압축 RAG 답변:")
print(compression_rag(test_query))
print("\n" + "=" * 70 + "\n")

print("4. Reranking RAG 답변:")
print(reranking_rag(test_query))
print("\n" + "=" * 70 + "\n")

print("5. 조합된 RAG 답변:")
print(combined_rag(test_query))